In [1]:
import os
import tensorflow as tf

In [2]:
# Directory containing the original images
original_dir = '../Dataset/disease_type/train_data/Tomato___Healthy'
# Directory where augmented images will be saved
augmented_dir = '../Dataset/disease_type/augmented_data/Tomato___Healthy'

# Ensure the output directory exists
os.makedirs(augmented_dir, exist_ok=True)

# Get a list of all image files in the original directory
image_files = tf.io.gfile.glob(os.path.join(original_dir, '*.jpg'))

# Apply augmentation (flipping, rotation, and noise) and save augmented images
for image_path in image_files:
    # Read the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Assuming RGB images
    
    # Apply flipping
    flipped_image = tf.image.flip_left_right(image)
    
    # Apply rotation
    rotated_image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))  # Rotate by 0, 90, 180, or 270 degrees
    
    # Add noise
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1, dtype=tf.float32)  # Adjust stddev as needed
    noise = tf.cast(noise, tf.uint8)  # Convert noise tensor to uint8
    noisy_flipped_image = tf.cast(flipped_image, tf.uint8) + noise
    noisy_rotated_image = tf.cast(rotated_image, tf.uint8) + noise

    # Clip pixel values to [0, 255]
    noisy_flipped_image = tf.clip_by_value(noisy_flipped_image, 0, 255)
    noisy_rotated_image = tf.clip_by_value(noisy_rotated_image, 0, 255)

    
    # Save the flipped image with noise
    filename = os.path.basename(image_path)
    flipped_image_path = os.path.join(augmented_dir, 'flipped_' + filename)
    tf.io.write_file(flipped_image_path, tf.image.encode_jpeg(noisy_flipped_image))
    
    # Save the rotated image with noise
    rotated_image_path = os.path.join(augmented_dir, 'rotated_' + filename)
    tf.io.write_file(rotated_image_path, tf.image.encode_jpeg(noisy_rotated_image))
